In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('ggplot')
from itertools import chain
import nltk
import sklearn
import scipy. stats
from sklearn.metrics import make_scorer
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RandomizedSearchCV
import sklearn_crfsuite
from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics
import pandas as pd
import numpy as np
from itertools import groupby
import pickle

In [2]:
with open('训练语料/relation extraction dataset/re_train.pkl', 'rb') as f:
    rtrain = pickle.load(f)
with open('训练语料/relation extraction dataset/re_test.pkl', 'rb') as f:
    rtest = pickle.load(f)
with open('训练语料/relation extraction dataset/re_valid.pkl', 'rb') as f:
    rvalid = pickle.load(f)

In [3]:
def tag2code(tags):
    code=[]
    #ent=()
    ents=[]
    n=0
    m=1
    for i in range(len(tags)):
        #print(i,tags[i][0])
        if tags[i][0]=='O':
            code.append('0')
        elif tags[i][0]=='B':
            n+=1
            code.append('T'+str(n))
            
        elif tags[i][0]=='I' and tags[i+1][0]=='I':
            code.append('T'+str(n))
            m+=1
        elif tags[i][0]=='I' and tags[i+1][0]!='I':
            code.append('T'+str(n))
            ent=(code[len(code)-1],tags[i][2:len(tags[i])],i-m, i)
            ents.append(ent)
    return ents,code

In [50]:
taglist=rtrain[0]['tags']
tag2code(taglist)

([('T1', 'NIHSS', 8, 9),
  ('T3', '1a_LOC', 12, 16),
  ('T5', '1b_LOCQuestions', 16, 22),
  ('T7', '1c_LOCCommands', 20, 28),
  ('T9', '2_BestGaze', 24, 34),
  ('T11', '3_Visual', 28, 39),
  ('T13', '4_FacialPalsy', 32, 45),
  ('T15', '5a_LeftArm', 36, 53),
  ('T17', '5b_RightArm', 40, 61),
  ('T19', '6a_LeftLeg', 44, 69),
  ('T21', '6b_RightLeg', 48, 77),
  ('T23', '7_LimbAtaxia', 52, 83),
  ('T25', '8_Sensory', 56, 88),
  ('T27', '9_BestLanguage', 60, 94),
  ('T29', '10_Dysarthria', 64, 99),
  ('T31', '11_ExtinctionInattention', 68, 106)],
 ['0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  'T1',
  'T1',
  '0',
  'T2',
  'T3',
  'T3',
  'T3',
  'T3',
  'T3',
  '0',
  'T4',
  'T5',
  'T5',
  'T5',
  'T5',
  '0',
  'T6',
  'T7',
  'T7',
  'T7',
  'T7',
  '0',
  'T8',
  'T9',
  'T9',
  'T9',
  'T9',
  '0',
  'T10',
  'T11',
  'T11',
  'T11',
  '0',
  'T12',
  'T13',
  'T13',
  'T13',
  'T13',
  '0',
  'T14',
  'T15',
  'T15',
  'T15',
  'T15',
  'T15',
  'T15',
  '0',
  'T16',
  '

In [ ]:
end2end_dict_list=[]
for i in rtrain:
    end2end_dict={}
    taglist=i['tags']
    end2end_dict['HADM_ID']=i[0]
    end2end_dict['token']=i[1]
    end2end_dict['tags']=i[2]
    end2end_dict['relations']=tag2code(taglist)[0]
    end2end_dict['code']=tag2code(taglist)[1]
    end2end_dict_list.append(end2end_dict)

In [93]:
def file2set(file):
    rset=[]
    pid=[]
    for i in file:
        subset=[]
        pid.append(i['HADM_ID'])
        code=tag2code(i['tags'])[1]

        for j in range(len(i['token'])):
            subset.append((i['token'][j],code[j],i['tags'][j]))
        rset.append(subset)
    return rset

In [94]:
rtrainset=file2set(rtrain)
rtestset=file2set(rtest)
rvalidset=file2set(rvalid)

training set

In [4]:
rtrainset=[]
trainpid=[]
for i in rtrain:
    trainset=[]
    trainpid.append(i['HADM_ID'])
    code=tag2code(i['tags'])[1]
    
    for j in range(len(i['token'])):
        trainset.append((i['token'][j],i['tags'][j],code[j]))
    rtrainset.append(trainset)
rtrainset[0:3]

[[(':', 'O', '0'),
  ('died', 'O', '0'),
  ('of', 'O', '0'),
  ('heart', 'O', '0'),
  ('problems', 'O', '0'),
  ('Physical', 'O', '0'),
  ('Exam', 'O', '0'),
  (':', 'O', '0'),
  ('NIH', 'B-NIHSS', 'T1'),
  ('SS', 'I-NIHSS', 'T1'),
  (':', 'O', '0'),
  ('2', 'B-Measurement', 'T2'),
  ('1a', 'B-1a_LOC', 'T3'),
  ('.', 'I-1a_LOC', 'T3'),
  ('Level', 'I-1a_LOC', 'T3'),
  ('of', 'I-1a_LOC', 'T3'),
  ('Consciousness', 'I-1a_LOC', 'T3'),
  (':', 'O', '0'),
  ('0', 'B-Measurement', 'T4'),
  ('1b', 'B-1b_LOCQuestions', 'T5'),
  ('.', 'I-1b_LOCQuestions', 'T5'),
  ('LOC', 'I-1b_LOCQuestions', 'T5'),
  ('questions', 'I-1b_LOCQuestions', 'T5'),
  (':', 'O', '0'),
  ('0', 'B-Measurement', 'T6'),
  ('1c', 'B-1c_LOCCommands', 'T7'),
  ('.', 'I-1c_LOCCommands', 'T7'),
  ('LOC', 'I-1c_LOCCommands', 'T7'),
  ('commands', 'I-1c_LOCCommands', 'T7'),
  (':', 'O', '0'),
  ('0', 'B-Measurement', 'T8'),
  ('2', 'B-2_BestGaze', 'T9'),
  ('.', 'I-2_BestGaze', 'T9'),
  ('Best', 'I-2_BestGaze', 'T9'),
  ('gaze',

testing set

In [5]:
rtestset=[]
testpid=[]
for i in rtest:
    testset=[]
    testpid.append(i['HADM_ID'])
    code=tag2code(i['tags'])[1]
    
    for j in range(len(i['token'])):
        testset.append((i['token'][j],i['tags'][j],code[j]))
    rtestset.append(testset)
rtestset[0:3]

[[('NIH', 'B-NIHSS', 'T1'),
  ('Stroke', 'I-NIHSS', 'T1'),
  ('Scale', 'I-NIHSS', 'T1'),
  ('Score', 'I-NIHSS', 'T1'),
  (':', 'O', '0'),
  ('10', 'B-Measurement', 'T2'),
  ('t', 'O', '0'),
  ('-', 'O', '0'),
  ('[', 'O', '0'),
  ('*', 'O', '0'),
  ('*', 'O', '0'),
  ('MD', 'O', '0'),
  ('Number', 'O', '0'),
  ('(', 'O', '0'),
  ('3', 'O', '0'),
  (')', 'O', '0'),
  ('6360', 'O', '0'),
  ('*', 'O', '0'),
  ('*', 'O', '0'),
  (']', 'O', '0'),
  (':', 'O', '0'),
  ('-', 'O', '0'),
  ('-', 'O', '0'),
  ('-', 'O', '0'),
  ('Yes', 'O', '0'),
  ('Time', 'O', '0'),
  ('t', 'O', '0'),
  ('-', 'O', '0'),
  ('PA', 'O', '0'),
  ('was', 'O', '0'),
  ('given', 'O', '0'),
  ('-', 'O', '0'),
  ('-', 'O', '0'),
  ('-', 'O', '0'),
  ('-', 'O', '0'),
  ('-', 'O', '0'),
  ('-', 'O', '0'),
  (':', 'O', '0'),
  ('-', 'O', '0'),
  ('-', 'O', '0'),
  ('-', 'O', '0'),
  ('-', 'O', '0'),
  ('-', 'O', '0'),
  ('-', 'O', '0'),
  ('(', 'O', '0'),
  ('24h', 'O', '0'),
  ('clock', 'O', '0'),
  (')', 'O', '0'),
  ('

validating set

In [6]:
rvalidset=[]
validpid=[]
for i in rvalid:
    validset=[]
    validpid.append(i['HADM_ID'])
    code=tag2code(i['tags'])[1]
    
    for j in range(len(i['token'])):
        validset.append((i['token'][j],i['tags'][j],code[j]))
    rvalidset.append(validset)
rvalidset[0:3]

[[('.', 'O', '0'),
  ('Neurology', 'O', '0'),
  (':', 'O', '0'),
  ('Patient', 'O', '0'),
  ('received', 'O', '0'),
  ('IV', 'O', '0'),
  ('TPA', 'O', '0'),
  ('in', 'O', '0'),
  ('ED', 'O', '0'),
  ('for', 'O', '0'),
  ('NIHSS', 'B-NIHSS', 'T1'),
  ('of', 'O', '0'),
  ('15', 'B-Measurement', 'T2'),
  ('and', 'O', '0'),
  ('admitted', 'O', '0'),
  ('to', 'O', '0'),
  ('Neurology', 'O', '0'),
  ('ICU', 'O', '0'),
  ('for', 'O', '0'),
  ('observation', 'O', '0'),
  ('and', 'O', '0'),
  ('post', 'O', '0')],
 [('NIHSS', 'B-NIHSS', 'T1'),
  (':', 'O', '0'),
  ('Best', 'B-2_BestGaze', 'T2'),
  ('gaze', 'I-2_BestGaze', 'T2'),
  ('-', 'O', '0'),
  ('forced', 'O', '0'),
  ('to', 'O', '0'),
  ('R', 'O', '0'),
  ('2', 'B-Measurement', 'T3'),
  ('Visual', 'B-3_Visual', 'T4'),
  ('-', 'O', '0'),
  ('Complete', 'O', '0'),
  ('L', 'O', '0'),
  ('hemianopia', 'O', '0'),
  ('2', 'B-Measurement', 'T5'),
  ('Facial', 'B-4_FacialPalsy', 'T6'),
  ('palsy', 'I-4_FacialPalsy', 'T6'),
  ('-', 'O', '0'),
  ('p

In [95]:
def word2features(sent,i):
    word=str(sent[i][0])
    #tag= sent[i][1] 
    
    features={
        'bias':1.0,
        'word.lower()':word.lower(),
        'word[-3:]':word[-3:],
        'word[-2:]':word[-2:],
        'word.isupper()':word.isupper(),
        'word.istitle()':word.istitle(),
        'word.isdigit()':word.isdigit(),
        #'tag-start':tag[0],
        #'tag-end':tag.split('-')[len(tag.split('-'))-1],
        
    }
    if i>0:
        word1=str(sent[i-1][0])
        #tag1=sent[i-1][1]
        features.update({
            '-1:word.lower()':word1.lower(),
            '-1:word.istitle()':word1.istitle(),
            '-1:word.isupper()':word1.isupper(),
            #'tag-start':tag1[0],
            #'tag-end':tag1.split('-')[len(tag1.split('-'))-1],
        })
    else:
        features['BOS']=True
        
    if i<len(sent)-1:
        word1=str(sent[i+1][0])
        #print(i,word1)
        #tag1=sent[i+1][1]
        features.update({
            '+1:word.lower()':word1.lower(),
            '+1:word.istitle()':word1.istitle(),
            '+1:word.isupper()':word1.isupper(),
            #'tag-start':tag1[0],
            #'tag-end':tag1.split('-')[len(tag1.split('-'))-1],
        })
    else:
        features['EOS']=True
    
    return features

def sent2features(sent):
    return [word2features(sent,i) for i in range(len(sent))]

def sent2labels(sent):
    return [label for token, postag, label in sent]

def sent2tokens(sent):
    return [token for token, postag, label in sent]

In [96]:
%%time
X_rtrain=[sent2features(s) for s in rtrainset]
y_rtrain=[sent2labels(s) for s in rtrainset]

X_rtest=[sent2features(s) for s in rtestset]
y_rtest=[sent2labels(s) for s in rtestset]

Wall time: 112 ms


In [97]:
X_rvalid=[sent2features(s) for s in rvalidset]
y_rvalid=[sent2labels(s) for s in rvalidset]

In [98]:
%%time
crf=sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    c1=0.1,
    c2=0.1,
    max_iterations=100,
    all_possible_transitions=True
)
crf.fit(X_rtrain, y_rtrain)

Wall time: 12.1 s


In [99]:
labels=list(crf.classes_)
labels.remove('O')
label_set=[x.split('-')[1] for x in labels]
labels

['B-NIHSS',
 'I-NIHSS',
 'B-Measurement',
 'B-1a_LOC',
 'I-1a_LOC',
 'B-1b_LOCQuestions',
 'I-1b_LOCQuestions',
 'B-1c_LOCCommands',
 'I-1c_LOCCommands',
 'B-2_BestGaze',
 'I-2_BestGaze',
 'B-3_Visual',
 'I-3_Visual',
 'B-4_FacialPalsy',
 'I-4_FacialPalsy',
 'B-5a_LeftArm',
 'I-5a_LeftArm',
 'B-5b_RightArm',
 'I-5b_RightArm',
 'B-6a_LeftLeg',
 'I-6a_LeftLeg',
 'B-6b_RightLeg',
 'I-6b_RightLeg',
 'B-7_LimbAtaxia',
 'I-7_LimbAtaxia',
 'B-8_Sensory',
 'I-8_Sensory',
 'B-9_BestLanguage',
 'I-9_BestLanguage',
 'B-10_Dysarthria',
 'I-10_Dysarthria',
 'B-11_ExtinctionInattention',
 'I-11_ExtinctionInattention',
 'B-56_Motor',
 'I-Measurement',
 'B-TemporalConstraint',
 'I-TemporalConstraint',
 'B-5_Motor',
 'I-5_Motor',
 'B-6_Motor',
 'I-6_Motor',
 'I-56_Motor']

In [100]:
y_rpred=crf.predict(X_rtest)
metrics.flat_f1_score(y_rtest, y_rpred, average='weighted',labels=labels)

0.9258587494062468

In [101]:
y_rpred

[['B-NIHSS',
  'I-NIHSS',
  'I-NIHSS',
  'I-NIHSS',
  'O',
  'B-Measurement',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'B-Measurement',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'B-NIHSS',
  'I-NIHSS',
  'I-NIHSS',
  'I-NIHSS',
  'O',
  'B-Measurement',
  'O',
  'B-1a_LOC',
  'I-1a_LOC',
  'I-1a_LOC',
  'I-1a_LOC',
  'I-1a_LOC',
  'O',
  'B-Measurement',
  'B-1b_LOCQuestions',
  'I-1b_LOCQuestions',
  'I-1b_LOCQuestions',
  'I-1b_LOCQuestions',
  'O',
  'B-Measurement',
  'B-1c_LOCCommands',
  'I-1c_LOCCommands',
  'I-1c_LOCCommands',

In [102]:
sorted_labels=sorted(labels, key=lambda name: (name[1:], name[0]))
print(metrics.flat_classification_report(y_rtest, y_rpred, labels=sorted_labels, digits=3))

D:\Anaconda\lib\site-packages\sklearn\utils\validation.py:71: FutureWarning: Pass labels=['B-10_Dysarthria', 'I-10_Dysarthria', 'B-11_ExtinctionInattention', 'I-11_ExtinctionInattention', 'B-1a_LOC', 'I-1a_LOC', 'B-1b_LOCQuestions', 'I-1b_LOCQuestions', 'B-1c_LOCCommands', 'I-1c_LOCCommands', 'B-2_BestGaze', 'I-2_BestGaze', 'B-3_Visual', 'I-3_Visual', 'B-4_FacialPalsy', 'I-4_FacialPalsy', 'B-56_Motor', 'I-56_Motor', 'B-5_Motor', 'I-5_Motor', 'B-5a_LeftArm', 'I-5a_LeftArm', 'B-5b_RightArm', 'I-5b_RightArm', 'B-6_Motor', 'I-6_Motor', 'B-6a_LeftLeg', 'I-6a_LeftLeg', 'B-6b_RightLeg', 'I-6b_RightLeg', 'B-7_LimbAtaxia', 'I-7_LimbAtaxia', 'B-8_Sensory', 'I-8_Sensory', 'B-9_BestLanguage', 'I-9_BestLanguage', 'B-Measurement', 'I-Measurement', 'B-NIHSS', 'I-NIHSS', 'B-TemporalConstraint', 'I-TemporalConstraint'] as keyword args. From version 0.25 passing these as positional arguments will result in an error
  FutureWarning)
D:\Anaconda\lib\site-packages\sklearn\metrics\_classification.py:1221: U

                            precision    recall  f1-score   support

           B-10_Dysarthria      0.879     0.927     0.903        55
           I-10_Dysarthria      1.000     0.987     0.993        77
B-11_ExtinctionInattention      1.000     0.894     0.944        47
I-11_ExtinctionInattention      1.000     0.953     0.976       149
                  B-1a_LOC      1.000     0.955     0.977        44
                  I-1a_LOC      1.000     0.985     0.993       137
         B-1b_LOCQuestions      1.000     0.956     0.977        45
         I-1b_LOCQuestions      1.000     0.958     0.979       120
          B-1c_LOCCommands      0.976     0.976     0.976        41
          I-1c_LOCCommands      0.991     1.000     0.995       109
              B-2_BestGaze      1.000     0.930     0.964        43
              I-2_BestGaze      1.000     0.965     0.982       114
                B-3_Visual      0.953     0.891     0.921        46
                I-3_Visual      0.981     0.954

In [103]:
%%time
# define fixed parameters and parameters to search
crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs', 
    max_iterations=100, 
    all_possible_transitions=True
)
params_space = {
    'c1': scipy.stats.expon(scale=0.5),
    'c2': scipy.stats.expon(scale=0.05),
}

# use the same metric for evaluation
f1_scorer = make_scorer(metrics.flat_f1_score, 
                        average='weighted', labels=labels)

# search
rs = RandomizedSearchCV(crf, params_space, 
                        cv=3, 
                        verbose=1, 
                        n_jobs=-1, 
                        n_iter=50, 
                        scoring=f1_scorer)
rs.fit(X_rtrain, y_rtrain)

D:\Anaconda\lib\site-packages\sklearn\base.py:213: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  FutureWarning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


Fitting 3 folds for each of 50 candidates, totalling 150 fits


[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:  1.4min
[Parallel(n_jobs=-1)]: Done 150 out of 150 | elapsed:  5.3min finished


Wall time: 5min 33s


In [104]:
crf = rs.best_estimator_
y_rpred_valid = crf.predict(X_rvalid)
print(metrics.flat_classification_report(
    y_rvalid, y_rpred_valid, labels=sorted_labels, digits=3
))

                            precision    recall  f1-score   support

           B-10_Dysarthria      0.909     0.800     0.851        25
           I-10_Dysarthria      0.923     0.960     0.941        25
B-11_ExtinctionInattention      0.947     0.783     0.857        23
I-11_ExtinctionInattention      1.000     0.945     0.972        55
                  B-1a_LOC      0.867     0.765     0.812        17
                  I-1a_LOC      0.976     0.952     0.964        42
         B-1b_LOCQuestions      1.000     0.864     0.927        22
         I-1b_LOCQuestions      1.000     1.000     1.000        40
          B-1c_LOCCommands      0.944     0.944     0.944        18
          I-1c_LOCCommands      0.973     1.000     0.986        36
              B-2_BestGaze      0.941     0.889     0.914        18
              I-2_BestGaze      0.925     0.949     0.937        39
                B-3_Visual      0.882     0.750     0.811        20
                I-3_Visual      0.947     0.973

D:\Anaconda\lib\site-packages\sklearn\utils\validation.py:71: FutureWarning: Pass labels=['B-10_Dysarthria', 'I-10_Dysarthria', 'B-11_ExtinctionInattention', 'I-11_ExtinctionInattention', 'B-1a_LOC', 'I-1a_LOC', 'B-1b_LOCQuestions', 'I-1b_LOCQuestions', 'B-1c_LOCCommands', 'I-1c_LOCCommands', 'B-2_BestGaze', 'I-2_BestGaze', 'B-3_Visual', 'I-3_Visual', 'B-4_FacialPalsy', 'I-4_FacialPalsy', 'B-56_Motor', 'I-56_Motor', 'B-5_Motor', 'I-5_Motor', 'B-5a_LeftArm', 'I-5a_LeftArm', 'B-5b_RightArm', 'I-5b_RightArm', 'B-6_Motor', 'I-6_Motor', 'B-6a_LeftLeg', 'I-6a_LeftLeg', 'B-6b_RightLeg', 'I-6b_RightLeg', 'B-7_LimbAtaxia', 'I-7_LimbAtaxia', 'B-8_Sensory', 'I-8_Sensory', 'B-9_BestLanguage', 'I-9_BestLanguage', 'B-Measurement', 'I-Measurement', 'B-NIHSS', 'I-NIHSS', 'B-TemporalConstraint', 'I-TemporalConstraint'] as keyword args. From version 0.25 passing these as positional arguments will result in an error
  FutureWarning)


In [105]:
def sent2tag(sent_list):
    sent2tag_list=[]
    for i in sent_list:
        sent2tag_list=sent2tag_list+i
    return sent2tag_list

def tag2entity(sent_list):
    tag2entity_list=[]
    for j in sent2tag(sent_list):
        if j=='O':
            tag2entity_list.append([j])
        elif j[0]=='B':
            tag2entity_list.append([j])
        elif j[0]=='I':
            tag2entity_list[len(tag2entity_list)-1].append(j)
    return tag2entity_list

#def tag2entity_pred(sent_list,y_pred):
#    tag2entity_pred_list=[]
#    x=0
#    for i in tag2entity(sent_list):
#        tag2entity_pred_list.append(sent2tag(y_pred)[x:x+len(i)])
#        x+=len(i)
#    return tag2entity_pred_list
        
def entity2label(tag2entity_list):
    entity2label_list=[]
    for k in tag2entity_list:
        #print(k)
        l=[]
        for m in k:
            lm=m.split('-')
            l.append(lm[len(lm)-1])
        if len(set(l))==1:
            entity2label_list.append(l[0])
        elif len(set(l))>1:
            entity2label_list.append(','.join(set(l)))  
            #print(','.join(set(l)))
    return entity2label_list

In [106]:
def tag2entity_pred(sent_list_test, sent_list_pred):
    tag2entity_pred_list=[]
    tag2entity_list=tag2entity(sent_list_test)
    sent2tag_pred_list=sent2tag(sent_list_pred)
    x=0
    for i in tag2entity_list:
        tag2entity_pred_list.append(sent2tag_pred_list[x:x+len(i)])
        x+=len(i)
    return tag2entity_pred_list

In [107]:
tag2entity_pred0_list=[]
tag2entity_list=tag2entity(y_rtest)
sent2tag_pred0_list=sent2tag(y_rpred)
x=0
for i in tag2entity_list:
    tag2entity_pred0_list.append(sent2tag_pred0_list[x:x+len(i)])
    x+=len(i)

In [108]:
y_test0_label=entity2label(tag2entity(y_rtest))

In [109]:
y_pred0_label=entity2label(tag2entity_pred0_list)

In [110]:
len(y_test0_label),len(y_pred0_label)

(11106, 11106)

In [111]:
y_test0_label

['NIHSS',
 'O',
 'Measurement',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'NIHSS',
 'O',
 'Measurement',
 'O',
 '1a_LOC',
 'O',
 'Measurement',
 '1b_LOCQuestions',
 'O',
 'Measurement',
 '1c_LOCCommands',
 'O',
 'Measurement',
 '2_BestGaze',
 'O',
 'Measurement',
 '3_Visual',
 'O',
 'Measurement',
 '4_FacialPalsy',
 'Measurement',
 '5a_LeftArm',
 'O',
 'Measurement',
 '5b_RightArm',
 'O',
 'Measurement',
 '6a_LeftLeg',
 'O',
 'Measurement',
 '6b_RightLeg',
 'O',
 'Measurement',
 '7_LimbAtaxia',
 'O',
 'Measurement',
 '8_Sensory',
 'O',
 'Measurement',
 '9_BestLanguag

In [112]:
y_test0_label

['NIHSS',
 'O',
 'Measurement',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'NIHSS',
 'O',
 'Measurement',
 'O',
 '1a_LOC',
 'O',
 'Measurement',
 '1b_LOCQuestions',
 'O',
 'Measurement',
 '1c_LOCCommands',
 'O',
 'Measurement',
 '2_BestGaze',
 'O',
 'Measurement',
 '3_Visual',
 'O',
 'Measurement',
 '4_FacialPalsy',
 'Measurement',
 '5a_LeftArm',
 'O',
 'Measurement',
 '5b_RightArm',
 'O',
 'Measurement',
 '6a_LeftLeg',
 'O',
 'Measurement',
 '6b_RightLeg',
 'O',
 'Measurement',
 '7_LimbAtaxia',
 'O',
 'Measurement',
 '8_Sensory',
 'O',
 'Measurement',
 '9_BestLanguag

calculate precision recall

precision: corrected predicted nihss entitiy / all entity predicted as nihss

In [34]:
TP_dict=dict()
PP_dict=dict()
RP_dict=dict()
for i in [TP_dict,PP_dict, RP_dict]:
    for j in label_set:
        i[j]=0

* TP: number of entities correctly predicted as A,
* PP: number of entities predicted as A, correct or not, 
* RP: real number of entities that is A

In [48]:
for i in range(len(y_test0_label)):
    if y_test0_label[i]==y_pred0_label[i] and y_test0_label[i]!='O':
        TP_dict[y_test0_label[i]]+=1

In [50]:
for i in y_pred0_label:
    if i in label_set:
        PP_dict[i]+=1

In [52]:
for i in y_test0_label:
    if i in label_set:
        RP_dict[i]+=1

In [57]:
precision_dict=dict()
recall_dict=dict()
f1_dict=dict()
for i in label_set:
    precision_dict[i]=TP_dict[i]/PP_dict[i]
    recall_dict[i]=TP_dict[i]/RP_dict[i]
    f1_dict[i]=2*(precision_dict[i]*recall_dict[i])/(precision_dict[i]+recall_dict[i])

In [58]:
precision_dict

{'10_Dysarthria': 0.9591836734693877,
 '11_ExtinctionInattention': 1.0,
 '1a_LOC': 0.9634146341463414,
 '1b_LOCQuestions': 0.9777777777777777,
 '1c_LOCCommands': 0.9880952380952381,
 '2_BestGaze': 0.9764705882352941,
 '3_Visual': 0.9545454545454546,
 '4_FacialPalsy': 0.9292929292929293,
 '56_Motor': 0.6666666666666666,
 '5_Motor': 0.9090909090909091,
 '5a_LeftArm': 0.9333333333333333,
 '5b_RightArm': 0.971830985915493,
 '6_Motor': 0.6875,
 '6a_LeftLeg': 0.9324324324324325,
 '6b_RightLeg': 0.9848484848484849,
 '7_LimbAtaxia': 0.9523809523809523,
 '8_Sensory': 0.9662921348314607,
 '9_BestLanguage': 0.9587628865979382,
 'Measurement': 0.9613679544015199,
 'NIHSS': 0.9424920127795527,
 'TemporalConstraint': 0.7142857142857143}

In [59]:
recall_dict

{'10_Dysarthria': 0.94,
 '11_ExtinctionInattention': 0.851063829787234,
 '1a_LOC': 0.9294117647058824,
 '1b_LOCQuestions': 0.9565217391304348,
 '1c_LOCCommands': 0.9540229885057471,
 '2_BestGaze': 0.9540229885057471,
 '3_Visual': 0.8842105263157894,
 '4_FacialPalsy': 0.8846153846153846,
 '56_Motor': 0.6666666666666666,
 '5_Motor': 0.7692307692307693,
 '5a_LeftArm': 0.9210526315789473,
 '5b_RightArm': 0.9452054794520548,
 '6_Motor': 0.7857142857142857,
 '6a_LeftLeg': 0.9452054794520548,
 '6b_RightLeg': 0.9558823529411765,
 '7_LimbAtaxia': 0.963855421686747,
 '8_Sensory': 0.8865979381443299,
 '9_BestLanguage': 0.93,
 'Measurement': 0.9638095238095238,
 'NIHSS': 0.9609120521172638,
 'TemporalConstraint': 0.759493670886076}

In [60]:
f1_dict

{'10_Dysarthria': 0.9494949494949495,
 '11_ExtinctionInattention': 0.9195402298850576,
 '1a_LOC': 0.9461077844311377,
 '1b_LOCQuestions': 0.967032967032967,
 '1c_LOCCommands': 0.9707602339181286,
 '2_BestGaze': 0.9651162790697674,
 '3_Visual': 0.9180327868852459,
 '4_FacialPalsy': 0.9064039408866995,
 '56_Motor': 0.6666666666666666,
 '5_Motor': 0.8333333333333333,
 '5a_LeftArm': 0.9271523178807947,
 '5b_RightArm': 0.9583333333333334,
 '6_Motor': 0.7333333333333334,
 '6a_LeftLeg': 0.9387755102040816,
 '6b_RightLeg': 0.9701492537313432,
 '7_LimbAtaxia': 0.9580838323353292,
 '8_Sensory': 0.9247311827956988,
 '9_BestLanguage': 0.9441624365482233,
 'Measurement': 0.962587190868738,
 'NIHSS': 0.9516129032258065,
 'TemporalConstraint': 0.736196319018405}

In [68]:
precision_df=pd.DataFrame(precision_dict, index=['precision'])
recall_df=pd.DataFrame(recall_dict, index=['recall'])
f1_df=pd.DataFrame(f1_dict, index=['f1'])
pd.concat([precision_df, recall_df,f1_df])

,TemporalConstraint,NIHSS,Measurement,1a_LOC,1b_LOCQuestions,1c_LOCCommands,2_BestGaze,3_Visual,4_FacialPalsy,5a_LeftArm,...,6a_LeftLeg,6b_RightLeg,7_LimbAtaxia,8_Sensory,9_BestLanguage,10_Dysarthria,11_ExtinctionInattention,5_Motor,6_Motor,56_Motor
precision,0.714286,0.942492,0.961368,0.963415,0.977778,0.988095,0.976471,0.954545,0.929293,0.933333,...,0.932432,0.984848,0.952381,0.966292,0.958763,0.959184,1.000000,0.909091,0.687500,0.666667
recall,0.759494,0.960912,0.963810,0.929412,0.956522,0.954023,0.954023,0.884211,0.884615,0.921053,...,0.945205,0.955882,0.963855,0.886598,0.930000,0.940000,0.851064,0.769231,0.785714,0.666667
f1,0.736196,0.951613,0.962587,0.946108,0.967033,0.970760,0.965116,0.918033,0.906404,0.927152,...,0.938776,0.970149,0.958084,0.924731,0.944162,0.949495,0.919540,0.833333,0.733333,0.666667


In [35]:
def overall_perf(test_label,pred_label):
    tp=0
    for i in range(len(test_label)):
        if test_label[i]==pred_label[i] and test_label[i]!='O':
            tp+=1
    while 'O' in test_label:
        test_label.remove('O')
    rp=len(test_label)
    
    while 'O' in pred_label:
        pred_label.remove('O')
    pp=len(pred_label)
    precision=tp/pp
    recall=tp/rp
    f1=2*(precision*recall)/(precision+recall)
    overall_df=pd.DataFrame(data={'precision':precision, 'recall':recall, 'f1':f1}, index=['overall'])
    return overall_df

In [36]:
def train_test(traindf,testdf):
    train_set=df2list(traindf)
    test_set=df2list(testdf)
    
    X_train=[sent2features(s) for s in train_set]
    y_train=[sent2labels(s) for s in train_set]
    X_test=[sent2features(s) for s in test_set]
    y_test=[sent2labels(s) for s in test_set]
    
    crf=sklearn_crfsuite.CRF(
        algorithm='lbfgs',
        c1=0.1,
        c2=0.1,
        max_iterations=100,
        all_possible_transitions=True
    )
    crf.fit(X_train, y_train)
    
    labels=list(crf.classes_)
    labels.remove('O')
    label_set=[x.split('-')[1] for x in labels]

    y_pred=crf.predict(X_test)
    f1_score=metrics.flat_f1_score(y_test, y_pred, average='weighted',labels=labels)
    print('f1:',f1_score)
    
    y_test_label=entity2label(tag2entity(y_test))
    y_pred_label=entity2label(tag2entity_pred(y_test, y_pred))
    
    TP_dict=dict()
    PP_dict=dict()
    RP_dict=dict()
    for i in [TP_dict,PP_dict, RP_dict]:
        for j in label_set:
            i[j]=0
    
    #print(y_test_label[3292])
    for l in range(len(y_test_label)):
        if y_test_label[l]==y_pred_label[l] and y_test_label[l]!='O':
            #print(l,y_test_label[l])
            TP_dict[y_test_label[l]]+=1
    
    for m in y_pred_label:
        if m in label_set:
            PP_dict[m]+=1
            
    for n in y_test_label:
        if n in label_set:
            RP_dict[n]+=1
            
    precision_dict=dict()
    recall_dict=dict()
    f1_dict=dict()
    for s in label_set:
        precision_dict[s]=TP_dict[s]/PP_dict[s]
        recall_dict[s]=TP_dict[s]/RP_dict[s]
        f1_dict[s]=2*(precision_dict[s]*recall_dict[s])/(precision_dict[s]+recall_dict[s])
    
    precision_df=pd.DataFrame(precision_dict, index=['precision'])
    recall_df=pd.DataFrame(recall_dict, index=['recall'])
    f1_df=pd.DataFrame(f1_dict, index=['f1'])
    performance=pd.concat([precision_df, recall_df,f1_df])
    performancet=pd.DataFrame(performance.values.T, index=performance.columns, columns=performance.index)
    #print('len test',len(y_test_label),'len pred',len(y_pred_label))
    overalldf=overall_perf(y_test_label,y_pred_label)
    performancet=performancet.append(overalldf)
    return performancet

In [37]:
train_test(traindf0,testdf0)

f1: 0.9631067254257678


,precision,recall,f1
TemporalConstraint,0.714286,0.759494,0.736196
NIHSS,0.942492,0.960912,0.951613
Measurement,0.961368,0.963810,0.962587
1a_LOC,0.963415,0.929412,0.946108
1b_LOCQuestions,0.977778,0.956522,0.967033
1c_LOCCommands,0.988095,0.954023,0.970760
2_BestGaze,0.976471,0.954023,0.965116
3_Visual,0.954545,0.884211,0.918033
4_FacialPalsy,0.929293,0.884615,0.906404
5a_LeftArm,0.933333,0.921053,0.927152


In [111]:
writer = pd.ExcelWriter('performance.xlsx')

for i,j,k in [(traindf0,testdf0,'fold0'),(traindf1,testdf1,'fold1'),(traindf2,testdf2,'fold2'),(traindf3,testdf3,'fold3'),(traindf4,testdf4,'fold4')]:
    df=train_test(i,j)
    df.to_excel(writer, sheet_name=k)
    print(k)
writer.save()

f1: 0.9631067254257678
fold0
f1: 0.9606946188770304
fold1
f1: 0.9606740884930965
fold2
f1: 0.963630808183747
fold3
f1: 0.9782743469769934
fold4


In [41]:
def train_test_vali(traindf,testdf,validf):
    train_set=df2list(traindf)
    test_set=df2list(testdf)
    vali_set=df2list(validf)
    
    X_train=[sent2features(s) for s in train_set]
    y_train=[sent2labels(s) for s in train_set]
    X_test=[sent2features(s) for s in test_set]
    y_test=[sent2labels(s) for s in test_set]
    X_vali=[sent2features(s) for s in vali_set]
    y_vali=[sent2labels(s) for s in vali_set]
    
    crf=sklearn_crfsuite.CRF(
        algorithm='lbfgs',
        c1=0.1,
        c2=0.1,
        max_iterations=100,
        all_possible_transitions=True
    )
    crf.fit(X_train, y_train)
    
    labels=list(crf.classes_)
    labels.remove('O')
    label_set=[x.split('-')[1] for x in labels]
    
    crf = sklearn_crfsuite.CRF(
        algorithm='lbfgs', 
        max_iterations=100, 
        all_possible_transitions=True
    )
    params_space = {
        'c1': scipy.stats.expon(scale=0.5),
        'c2': scipy.stats.expon(scale=0.05),
    }
    
    # use the same metric for evaluation
    f1_scorer = make_scorer(metrics.flat_f1_score, 
                            average='weighted', labels=label_set)

    # search
    rs = RandomizedSearchCV(crf, params_space, 
                            #cv=3, 
                            verbose=1, 
                            n_jobs=-1, 
                            n_iter=50, 
                            scoring=f1_scorer)
    rs.fit(X_train, y_train)
    crf=rs.best_estimator_
    y_pred_vali=crf.predict(X_vali)

    #y_pred=crf.predict(X_test)
    #f1_score=metrics.flat_f1_score(y_test, y_pred, average='weighted',labels=labels)
    #print('f1:',f1_score)
    
    y_vali_label=entity2label(tag2entity(y_vali))
    y_pred_vali_label=entity2label(tag2entity_pred(y_vali, y_pred_vali))
    
    performance=perfm(y_vali_label,y_pred_vali_label)
    overalldf=overall_perf(y_vali_label,y_pred_vali_label)
    performance=performance.append(overalldf)
    
    return performance

In [42]:
train_test_vali(traindf1,testdf1,validf)

D:\Anaconda\lib\site-packages\sklearn\base.py:213: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  FutureWarning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


Fitting 5 folds for each of 50 candidates, totalling 250 fits


[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:  1.4min
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:  6.8min
[Parallel(n_jobs=-1)]: Done 250 out of 250 | elapsed:  9.3min finished


,precision,recall,f1
TemporalConstraint,0.148148,0.250000,0.186047
NIHSS,0.967949,0.909639,0.937888
Measurement,0.916364,0.945000,0.930462
1a_LOC,1.000000,0.954545,0.976744
1b_LOCQuestions,1.000000,0.955556,0.977273
1c_LOCCommands,1.000000,0.975610,0.987654
2_BestGaze,1.000000,0.953488,0.976190
3_Visual,0.976190,0.891304,0.931818
4_FacialPalsy,0.859649,0.859649,0.859649
5a_LeftArm,1.000000,0.809524,0.894737


In [43]:
y=0
for i,j,k in [(traindf0,testdf0,'fold0'),(traindf1,testdf1,'fold1'),(traindf2,testdf2,'fold2'),(traindf3,testdf3,'fold3'),(traindf4,testdf4,'fold4')]:
    print(y)
    if y==0:
        df=train_test_vali(i,j,validf)
        y+=1
    else:
        dfy=train_test_vali(i,j,validf)
        df=pd.concat([df,dfy], axis=1, join_axes=[df.index])
        y+=1
df

0


D:\Anaconda\lib\site-packages\sklearn\base.py:213: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  FutureWarning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


Fitting 5 folds for each of 50 candidates, totalling 250 fits


[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:  7.7min
[Parallel(n_jobs=-1)]: Done 250 out of 250 | elapsed: 10.3min finished


1
Fitting 5 folds for each of 50 candidates, totalling 250 fits


...

3
Fitting 5 folds for each of 50 candidates, totalling 250 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:  1.6min
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:  7.8min
[Parallel(n_jobs=-1)]: Done 250 out of 250 | elapsed: 10.4min finished


4
Fitting 5 folds for each of 50 candidates, totalling 250 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:  1.6min
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:  7.7min
[Parallel(n_jobs=-1)]: Done 250 out of 250 | elapsed: 10.3min finished


,precision,recall,f1,precision,recall,f1,precision,recall,f1,precision,recall,f1,precision,recall,f1
TemporalConstraint,0.155556,0.218750,0.181818,0.140351,0.250000,0.179775,0.160000,0.250000,0.195122,0.200000,0.187500,0.193548,0.121951,0.156250,0.136986
NIHSS,0.956250,0.921687,0.938650,0.967949,0.909639,0.937888,0.955975,0.915663,0.935385,0.950000,0.915663,0.932515,0.943396,0.903614,0.923077
Measurement,0.899289,0.948750,0.923358,0.914458,0.948750,0.931288,0.904988,0.952500,0.928136,0.885111,0.943750,0.913491,0.908544,0.943750,0.925812
1a_LOC,1.000000,0.954545,0.976744,1.000000,0.954545,0.976744,1.000000,0.954545,0.976744,1.000000,0.954545,0.976744,1.000000,0.954545,0.976744
1b_LOCQuestions,1.000000,0.933333,0.965517,1.000000,0.955556,0.977273,1.000000,0.955556,0.977273,1.000000,0.955556,0.977273,1.000000,0.955556,0.977273
1c_LOCCommands,1.000000,0.951220,0.975000,1.000000,0.975610,0.987654,1.000000,0.975610,0.987654,1.000000,0.975610,0.987654,1.000000,0.975610,0.987654
2_BestGaze,0.911111,0.953488,0.931818,1.000000,0.953488,0.976190,1.000000,0.953488,0.976190,0.872340,0.953488,0.911111,1.000000,0.953488,0.976190
3_Visual,0.931818,0.891304,0.911111,0.976190,0.891304,0.931818,0.952381,0.869565,0.909091,1.000000,0.891304,0.942529,0.836735,0.891304,0.863158
4_FacialPalsy,0.890909,0.859649,0.875000,0.924528,0.859649,0.890909,0.847458,0.877193,0.862069,0.862069,0.877193,0.869565,0.784615,0.894737,0.836066
5a_LeftArm,0.921053,0.833333,0.875000,1.000000,0.809524,0.894737,1.000000,0.809524,0.894737,1.000000,0.833333,0.909091,0.853659,0.833333,0.843373


In [44]:
df.to_csv('performance_vali.csv')

In [35]:
%%time
# define fixed parameters and parameters to search
crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs', 
    max_iterations=100, 
    all_possible_transitions=True
)
params_space = {
    'c1': scipy.stats.expon(scale=0.5),
    'c2': scipy.stats.expon(scale=0.05),
}

# use the same metric for evaluation
f1_scorer = make_scorer(metrics.flat_f1_score, 
                        average='weighted', labels=labels)

# search
rs = RandomizedSearchCV(crf, params_space, 
                        cv=3, 
                        verbose=1, 
                        n_jobs=-1, 
                        n_iter=50, 
                        scoring=f1_scorer)
rs.fit(X_train0, y_train0)
crf=rs.best_estimator_
y_pred_t=crf.predict(X_vali)

D:\Anaconda\lib\site-packages\sklearn\base.py:213: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  FutureWarning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


Fitting 3 folds for each of 50 candidates, totalling 150 fits


[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:  1.4min
[Parallel(n_jobs=-1)]: Done 150 out of 150 | elapsed:  5.5min finished


Wall time: 5min 43s


In [40]:
def perfm(y_real,y_pred):
    TP_dict=dict()
    PP_dict=dict()
    RP_dict=dict()
    for i in [TP_dict,PP_dict, RP_dict]:
        for j in label_set:
            i[j]=0
    
    for i in range(len(y_real)):
        if y_real[i]==y_pred[i] and y_real[i]!='O':
            TP_dict[y_real[i]]+=1
    for i in y_pred:
        if i in label_set:
            PP_dict[i]+=1
    #print(PP_dict)
    for i in y_real:
        if i in label_set:
            RP_dict[i]+=1
            
    precision_dict=dict()
    recall_dict=dict()
    f1_dict=dict()
    for i in label_set:
        if PP_dict[i]==0:
            precision_dict[i]=0
        else:   
            precision_dict[i]=TP_dict[i]/PP_dict[i]
        if RP_dict[i]==0:
            recall_dict[i]=0
        else:
            recall_dict[i]=TP_dict[i]/RP_dict[i]
        if (precision_dict[i]+recall_dict[i])==0:
            f1_dict[i]=0
        else:
            f1_dict[i]=2*(precision_dict[i]*recall_dict[i])/(precision_dict[i]+recall_dict[i])
    
    precision_df=pd.DataFrame(precision_dict, index=['precision'])
    recall_df=pd.DataFrame(recall_dict, index=['recall'])
    f1_df=pd.DataFrame(f1_dict, index=['f1'])
    perf_df=pd.concat([precision_df, recall_df,f1_df])
    perf_dft=pd.DataFrame(perf_df.values.T, index=perf_df.columns, columns=perf_df.index)
    return perf_dft

In [74]:
y_vali_list=entity2label(tag2entity(y_vali))
y_pred_t_list=entity2label(tag2entity_pred(y_vali, y_pred_t))

In [75]:
perfm(y_vali_list,y_pred_t_list)

,precision,recall,f1
TemporalConstraint,0.162791,0.218750,0.186667
NIHSS,0.956250,0.921687,0.938650
Measurement,0.898465,0.951250,0.924104
1a_LOC,1.000000,0.954545,0.976744
1b_LOCQuestions,1.000000,0.933333,0.965517
1c_LOCCommands,1.000000,0.951220,0.975000
2_BestGaze,0.911111,0.953488,0.931818
3_Visual,0.911111,0.891304,0.901099
4_FacialPalsy,0.836066,0.894737,0.864407
5a_LeftArm,0.921053,0.833333,0.875000


In [ ]:
%%time
crf=sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    max_iterations=100,
    all_possible_transitions=True
)
params_space={
    'c1':scipy.stats.expon(scale=0.5),
    'c2':scipy.stats.expon(scale=0.05),
}

f1_scorer=make_scorer(metrics.flat_f1_score,average='weighted',labels=labels)

rs=RandomizedSearchCV(crf,params_space, cv=3, verbose=1, n_jobs=-1, n_iter=50, scoring=f1_scorer)

rs.fit(X_train, y_train)

In [ ]:
print('best params:', rs.best_params_)
print('best cv score', rs.best_score_)
print('model size: {:0.2f}M'.format(rs.best_estimator_.size_/1000000))

check parameter space

In [ ]:
[s for s in rs.cv_results_]

In [ ]:
#_x = [s.parameters['param_c1'] for s in rs.cv_results_]
#_y = [s.parameters['param_c2'] for s in rs.cv_results_]
#_c = [s.mean_validation_score for s in rs.cv_results_]
_x=rs.cv_results_['param_c1']
_y=rs.cv_results_['param_c2']
_c=rs.cv_results_['mean_test_score']

fig = plt.figure()
fig.set_size_inches(12, 12)
ax = plt.gca()
ax.set_yscale('log')
ax.set_xscale('log')
ax.set_xlabel('C1')
ax.set_ylabel('C2')
ax.set_title("Randomized Hyperparameter Search CV Results (min={:0.3}, max={:0.3})".format(
    min(_c), max(_c)
))

ax.scatter(_x, _y, c=_c, s=60, alpha=0.9, edgecolors=[0,0,0])

print("Dark blue => {:0.4}, dark red => {:0.4}".format(min(_c), max(_c)))

check best estimator on test data

In [ ]:
crf=rs.best_estimator_
y_pred=crf.predict(X_test)
print(metrics.flat_classification_report(y_test,y_pred,labels=sorted_labels,digits=3))

check what classifier learned

In [ ]:
from collections import Counter

def print_transitions(trans_features):
    for (label_from, label_to), weight in trans_features:
        print('%-6s -> %-7s %0.6f' % (label_from, label_to,weight))
        
print('top likely transitions:')
print_transitions(Counter(crf.transition_features_).most_common(20))

print('\nTop unlikely transitions:')
print_transitions(Counter(crf.transition_features_).most_common()[-20:])

check state features:

In [ ]:
def print_state_features(state_features):
    for (attr, label), weight in state_features:
        print('%0.6f %-8s %s' % (weight, label, attr))
        
print("Top positive:")
print_state_features(Counter(crf.state_features_).most_common(30))

print("\nTop negative:")
print_state_features(Counter(crf.state_features_).most_common()[-30:])  